In [1]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left=10,
    truncated_length_right=100,
    filter_low_freq=2
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10815.77it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5861.10it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 3460206.26it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11302.19it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 6104.96it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 946791.29it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 10139.60it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x113058198>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x13c2c6c88>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
fixed_padding_callback = mz.dataloader.callbacks.BasicPadding(
    fixed_length_left=10,
    fixed_length_right=100
)

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=fixed_padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=fixed_padding_callback
)

In [8]:
model = mz.models.ArcI()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['left_filters'] = [128]
model.params['left_kernel_sizes'] = [3]
model.params['left_pool_sizes'] = [4]
model.params['right_filters'] = [128]
model.params['right_kernel_sizes'] = [3]
model.params['right_pool_sizes'] = [4]
model.params['conv_activation_func'] = 'relu'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 100
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.9

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ArcI(
  (embedding): Embedding(16675, 300)
  (conv_left): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (conv_right): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dropout): Dropout(p=0.9, inplace=False)
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3456, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  5578959


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.943]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4243 - normalized_discounted_cumulative_gain@5(0.0): 0.5012 - mean_average_precision(0.0): 0.4553



[Iter-204 Loss-0.810]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5275 - normalized_discounted_cumulative_gain@5(0.0): 0.6081 - mean_average_precision(0.0): 0.5592



[Iter-306 Loss-0.746]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.55 - normalized_discounted_cumulative_gain@5(0.0): 0.6125 - mean_average_precision(0.0): 0.5728



[Iter-408 Loss-0.655]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5565 - normalized_discounted_cumulative_gain@5(0.0): 0.6078 - mean_average_precision(0.0): 0.57



[Iter-510 Loss-0.579]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5901 - normalized_discounted_cumulative_gain@5(0.0): 0.6503 - mean_average_precision(0.0): 0.6112



[Iter-612 Loss-0.506]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.577 - normalized_discounted_cumulative_gain@5(0.0): 0.6432 - mean_average_precision(0.0): 0.598



[Iter-714 Loss-0.461]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5772 - normalized_discounted_cumulative_gain@5(0.0): 0.6455 - mean_average_precision(0.0): 0.5895



[Iter-816 Loss-0.390]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5804 - normalized_discounted_cumulative_gain@5(0.0): 0.6386 - mean_average_precision(0.0): 0.5932



[Iter-918 Loss-0.335]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5884 - normalized_discounted_cumulative_gain@5(0.0): 0.645 - mean_average_precision(0.0): 0.6048



[Iter-1020 Loss-0.252]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5699 - normalized_discounted_cumulative_gain@5(0.0): 0.6273 - mean_average_precision(0.0): 0.5891

Cost time: 530.6298129558563s
